In [37]:
import pandas as pd
plates, normals = pd.read_csv('plates.csv',header=None), pd.read_csv('normals.csv',header=None)

def get_total():
    counter = int(input('Тип счетчика одноставочный (1), двуставочный (2) или трехставочный (3)? Если счетчика нет - 0'))
    plate = int(input('Помещение оборудовано газовой (1) или электрической (0) плитой?'))
    people = int(input('Сколько человек проживает в помещении?'))
    priv = int(input('В помещении проживают граждане льготной категории? 1 - да, 0 - нет'))
    water = int(input('Есть ли в помещении электроводонагреватели? 1 - да, 0 - нет'))
    rooms = int(input('Сколько жилых комнат в помещении?'))
    isend = int(input('Месяц закончился? Если да, то 1, если нет - 0'))
    n, r, dif = get_norm(priv,people), get_plate(plate), 0
    if counter < 2:
        q = float(input('Количество потребленной энергии в кВт (см. на счетчике)'))
        if counter == 1:
            if q > n: #для одноставочного счетчика, если потреблено > соц.нормы
                total = (n*r[0][0] + (q-n)*r[0][1], {'Текущий тариф': r[0][1]}) 
            else: #для одноставочного счетчика, если потреблено < соц.нормы
                total = (q*r[0][0], {'Текущий тариф': r[0][0]})
                dif = q-n
        else: #если счетчика нет
            total = (nr*r[0][0]*people, {'Текущий тариф': r[0][0], 'Текущий норматив': nr})
    elif counter == 2: #для двуставочного счетчика
        q_day = float(input('Количество энергии в кВт, потребленной ДНЕМ'))
        q_night = float(input('Количество энергии в кВт, потребленной НОЧЬЮ'))
        if (q_day + q_night) > n:
            M1, v1 = numpy.array([[1., 1.], [q_day, -q_night]]), numpy.array([n,0.])
            counts = numpy.linalg.solve(M1, v1)
            total = (counts[0]*r[1][0] + counts[1]*r[2][0] + (q_day-counts[0])*r[1][1] + (q_night-counts[1])*r[2][1],
                     {'Текущий ночной тариф': r[2][1], 'Текущий дневной тариф': r[1][1]})
        else:
            total = (q_day*r[1][0] + q_night*r[2][0], {'Текущий ночной тариф': r[2][0], 'Текущий дневной тариф': r[1][0]})
            dif = q_day + q_night - n
            
    elif counter == 3: #для трехставочного счетчика
        q_peak = float(input('Количество энергии в кВт, потребленной в ПИКОВЫЕ ЧАСЫ'))
        q_semipeak = float(input('Количество энергии в кВт, потребленной в ПОЛУПИКОВЫЕ ЧАСЫ'))
        q_night = float(input('Количество энергии в кВт, потребленной НОЧЬЮ'))
        if (q_peak + q_semipeak + q_night) > n:
            M1, v1 = numpy.array([[1., 1., 1.], [q_semipeak, q_night-q_peak, -q_semipeak],
                    [q_night+q_semipeak, -q_peak, -q_peak], [q_night, q_night, -q_peak-q_semipeak]]), numpy.array([n,0.,0.,0.])
            counts = numpy.linalg.solve(M1, v1)
            total = (counts[0]*r[3][0] + counts[1]*r[4][0] + counts[2]*r[5][0] + (q_peak-counts[0])*r[3][1] + 
                     (q_semipeak-counts[1])*r[4][1] + (q_night-counts[2])*r[5][1], {'Текущий тариф пиковой зоны': r[3][1], 
                                                                                    'Текущий тариф полупиковой зоны': r[4][1],
                                                                                   'Текущий тариф ночной зоны': r[5][1]})
        else:
            total = (q_peak*r[3][0] + q_semipeak*r[4][0] + q_night*r[5][0], {'Текущий тариф пиковой зоны': r[3][0], 
                                                                                    'Текущий тариф полупиковой зоны': r[4][0],
                                                                                   'Текущий тариф ночной зоны': r[5][0]})
            dif = q_peak + q_semipeak + q_night - n
    print('Сумма к оплате:',total[0],'рублей')
    if isend == 0:
        for key in total[1]:
            print(key, total[1][key])
        print('Остаток электроэнергии в рамках социальной нормы:',dif,'кВт')
    dif = 0
    
            
def get_plate(plate): #обращаемся к части бд
    if plate == 1: #для помещений с газовыми плитами
        return plates[:6]
    else: #для помещений с электроплитами
        return plates[6:]

def get_norm(priv,people): #определяем соц.норму
    if priv == 1: #для льготников
        return 75
    else: #если льготников нет
        return 85+(50*(people-1))
    
def get_normal(normals,people,rooms,plate,water): #берем инфу из бд по нормативам
    if rooms > 4: 
        r = rooms 
    else: 
        r = 4
    if people < 6:
        p = people
    else:
        p = 5
    if water == 0 and plate == 1: #для квартир с газовыми плитами, без электроводонагревателей
        return normals[p-1][r-1]
    if water == 0 and plate == 0: #для квартир с электроплитами, без электроводонагревателей
        return normals[p-1][r+3]
    if water == 1 and plate == 1: #для квартир с газовыми плитами, с электроводонагревателем
        return normals[p-1][r+7]
    if water == 1 and plate == 0: #для квартир с электроплитами, с электроводонагревателем
        return normals[p-1][r+11]

In [38]:
get_total()

Тип счетчика одноставочный (1), двуставочный (2) или трехставочный (3)? Если счетчика нет - 02
Помещение оборудовано газовой (1) или электрической (0) плитой?1
Сколько человек проживает в помещении?5
В помещении проживают граждане льготной категории? 1 - да, 0 - нет0
Есть ли в помещении электроводонагреватели? 1 - да, 0 - нет1
Сколько жилых комнат в помещении?3
Месяц закончился? Если да, то 1, если нет - 00
Количество энергии в кВт, потребленной ДНЕМ34
Количество энергии в кВт, потребленной НОЧЬЮ45


KeyError: 2